In [1]:
import sys
sys.path.append('../')
from pathlib import Path
import pandas as pd
import numpy as np

from scripts.base import Feature, get_arguments, generate_features
Feature.dir = "F:/Kaggle/ventilator-pressure-prediction/data/input/features"

In [2]:
INPUT_DIR = Path("F:/Kaggle/ventilator-pressure-prediction/data/input/")
OUTPUT_DIR = Path('F:/Kaggle/ventilator-pressure-prediction/data/output/')
FEAT_DIR = Path("F:/Kaggle/ventilator-pressure-prediction/data/input/features/")

def get_train_test():
    _train = pd.read_csv(INPUT_DIR / "train.csv")
    _test = pd.read_csv(INPUT_DIR / "test.csv")
    return _train, _test

def load_datasets(feats):
    dfs = [pd.read_feather(FEAT_DIR / f'{f}_train.ftr') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    dfs = [pd.read_feather(FEAT_DIR / f'{f}_test.ftr') for f in feats]
    X_test = pd.concat(dfs, axis=1)
    return X_train, X_test

In [ ]:
class Base(Feature):
    """結合用のデータ"""
    def create_features(self):
        self.train = train[['id', 'breath_id', 'time_step', 'u_in', 'u_out', 'pressure', 'fold']].copy()
        self.test = test[['id', 'breath_id', 'time_step', 'u_in', 'u_out']].copy()
        
train, test = get_train_test()
train_v2 = pd.read_csv(INPUT_DIR / "train_v2.csv")
train = pd.concat([train, train_v2[["fold"]]],axis=1)
#Base().run().save()

In [ ]:
class Area_cumsum(Feature):
    """u_inの下の面積っぽいけどなんか式違いそう"""
    def create_features(self):
        def create(df):
            df['area_cumsum'] = df['time_step'] * df['u_in']
            df['area_cumsum'] = df.groupby('breath_id')['area_cumsum'].cumsum()
            return df
        cols = ['area_cumsum']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
Area_cumsum().run().save()

In [ ]:
class Area(Feature):
    """その時点までのu_in曲線の積分、変化量、時間の変化量"""
    def create_features(self):
        def create(df):
            df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
            df['delta'] = df['time_delta'] * df['u_in']
            df['area'] = df.groupby('breath_id')['delta'].cumsum()
            df = df.fillna(0)
            return df
        cols = ['time_delta','delta','area']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
#Area().run().save()

In [ ]:
class Cross(Feature):
    """よくわからない"""
    def create_features(self):
        self.train['cross']  = train['u_in']*train['u_out'] 
        self.train['cross2'] = train['time_step']*train['u_out']
        self.test['cross']  = test['u_in']*test['u_out']
        self.test['cross2'] = test['time_step']*test['u_out']
        
train, test = get_train_test()
Cross().run().save()

In [ ]:
class U_in_cumsum_mean(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()# 累積和
            df['one'] = 1
            df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
            df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
            return df
        self.train = create(train)[['u_in_cumsum', 'u_in_cummean']]
        self.test = create(test)[['u_in_cumsum', 'u_in_cummean']]
        
train, test = get_train_test()
U_in_cumsum_mean().run().save()

In [ ]:
class U_in_Lag(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_in_lag'] = df.groupby('breath_id')['u_in'].shift(1)
            df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
            df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
            df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
            df['u_in_lag_back'] = df.groupby('breath_id')['u_in'].shift(-1)
            df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
            df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
            df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
            df = df.fillna(0)
            return df
        cols = ['u_in_lag','u_in_lag2','u_in_lag3','u_in_lag4','u_in_lag_back','u_in_lag_back2','u_in_lag_back3','u_in_lag_back4']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_in_Lag().run().save()

In [ ]:
class U_out_Lag(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_out_lag'] = df.groupby('breath_id')['u_out'].shift(1)
            df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
            df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
            df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
            df['u_out_lag_back'] = df.groupby('breath_id')['u_out'].shift(-1)
            df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
            df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
            df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
            df = df.fillna(0)
            return df
        cols = ['u_out_lag','u_out_lag2','u_out_lag3','u_out_lag4','u_out_lag_back','u_out_lag_back2','u_out_lag_back3','u_out_lag_back4']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_out_Lag().run().save()

In [ ]:
class RC_OHE(Feature):
    """
    RとCのOHE
    """
    def create_features(self):
        def create(df):
            df['R'] = df['R'].astype(str)
            df['C'] = df['C'].astype(str)
            df['RC'] = df['R']+df['C']
            df = pd.get_dummies(df)
            return df
        cols = ['R_20', 'R_5', 'R_50', 'C_10', 'C_20', 'C_50', 'RC_2010', 'RC_2020', 'RC_2050', 'RC_5010', 'RC_5020', 'RC_5050', 'RC_510', 'RC_520', 'RC_550']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
RC_OHE().run().save()

In [ ]:
class U_out_stat(Feature):
    """
    u_outが0のときと1のときのu_inのmean,std,max
    breath_idごとに値はおなじになる
    """
    def create_features(self):
        def create(df):
            df = df.merge(
                df[df["u_out"]==0].groupby('breath_id')['u_in'].agg(["mean", "std", "max"]).add_prefix("u_out0_").reset_index(),
                on="breath_id"
                )
            df = df.merge(
                df[df["u_out"]==1].groupby('breath_id')['u_in'].agg(["mean", "std", "max"]).add_prefix("u_out1_").reset_index(),
                on="breath_id"
                )
            return df
        cols = ['u_out0_mean', 'u_out0_max', 'u_out0_std', 'u_out1_mean', 'u_out1_max', 'u_out1_std']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_out_stat().run().save()

In [ ]:
class U_inout_max(Feature):
    """
    u_out関係なくu_inのmaxとu_outのmax
    """
    def create_features(self):
        def create(df):
            df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
            df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
            return df
        cols = ['breath_id__u_in__max', 'breath_id__u_out__max']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_inout_max().run().save()

In [ ]:
"""
breath_time は　Areaのtime_deltaと同じ
u_in_timeは後から　u_in changeを作る
"""
# class Time(Feature):
#     """
#     """
#     def create_features(self):
#         def create(df):
#             # breath_time
#             df['breath_time'] = df['time_step'] - df['time_step'].shift(1)
#             df.loc[df['time_step'] == 0, 'breath_time'] = 0
#             # u_in_time
#             df['u_in_time'] = df['u_in'] - df['u_in'].shift(1)
#             df.loc[df['time_step'] == 0, 'u_in_time'] = 0
#             return df
#         cols = ['breath_time', 'u_in_time']
#         self.train = create(train)[cols]
#         self.test = create(test)[cols]

# train, test = get_train_test()
# Time().run().save()

In [ ]:
class U_in_Lag_Diff(Feature):
    """
    """
    def create_features(self):
        def create(df):
            df["u_in_lag1_diff"] = df["u_in"] - df["u_in_lag"]
            df["u_in_lag2_diff"] = df["u_in"] - df["u_in_lag2"]
            df["u_in_lag3_diff"] = df["u_in"] - df["u_in_lag3"]
            df["u_in_lag4_diff"] = df["u_in"] - df["u_in_lag4"]
            return df
        cols = ['u_in_lag1_diff', 'u_in_lag2_diff','u_in_lag3_diff', 'u_in_lag4_diff']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        display(self.train)

feats = ['Base', 'U_in_Lag']
train, test = load_datasets(feats)
display(train)
U_in_Lag_Diff().run().save()

In [ ]:
class U_in_Rolling(Feature):
    """
    """
    def create_features(self):
        def create(df):
            df["u_in_rolling_mean2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).mean()["u_in"].reset_index(drop=True)
            df["u_in_rolling_mean4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).mean()["u_in"].reset_index(drop=True)
            df["u_in_rolling_mean10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).mean()["u_in"].reset_index(drop=True)
            
            df["u_in_rolling_max2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_max4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_max10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).std()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).std()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).std()["u_in"].reset_index(drop=True)
            for col in df.columns:
                df[col] = df[col].fillna(df[col].mean())
            return df
        cols = ['u_in_rolling_mean2', 'u_in_rolling_mean4','u_in_rolling_mean10', 'u_in_rolling_max2', 'u_in_rolling_max4', 'u_in_rolling_max10',
                'u_in_rolling_min2', 'u_in_rolling_min4', 'u_in_rolling_min10', 'u_in_rolling_std2', 'u_in_rolling_std4', 'u_in_rolling_std10']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        display(self.train)

train, test = get_train_test()
U_in_Rolling().run().save()

In [ ]:
class U_in_delta(Feature):
# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/273974
    """
    """
    def create_features(self):
        def create(df):
            df['u_in_change']= df['u_in'].shift(-1, fill_value=0)-df['u_in']
            df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
            df['area_u_in_abs']=df['u_in_change']*df['time_delta']
            df['uin_in_time']=df['u_in_change']/df['time_delta']
            df.loc[df["time_step"] == 0, ["uin_in_time"]] = 0
            return df
            
        cols = ['u_in_change', 'area_u_in_abs', 'uin_in_time']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_in_delta().run().save()

In [3]:
feats = ['base','Area', 'U_out_Lag']
train, test = load_datasets(feats)
train.head(50)

,id,breath_id,time_step,u_in,u_out,pressure,fold,time_delta,delta,area,u_out_lag,u_out_lag2,u_out_lag3,u_out_lag4,u_out_lag_back,u_out_lag_back2,u_out_lag_back3,u_out_lag_back4
0,1,1,0.000000,0.083334,0,5.837492,4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,0.033652,18.383041,0,5.907794,4,0.033652,0.618632,0.618632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,0.067514,22.509278,0,7.876254,4,0.033862,0.762212,1.380843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,0.101542,22.808822,0,11.742872,4,0.034028,0.776134,2.156978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,0.135756,25.355850,0,12.234987,4,0.034213,0.867507,3.024485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,1,0.169698,27.259866,0,12.867706,4,0.033942,0.925260,3.949746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,1,0.203708,27.127486,0,14.695562,4,0.034010,0.922610,4.872356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,1,0.237723,26.807732,0,15.890699,4,0.034015,0.911857,5.784213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,1,0.271776,27.864715,0,15.539188,4,0.034054,0.948900,6.733113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,1,0.305732,28.313036,0,15.750094,4,0.033955,0.961379,7.694491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train, test = get_train_test()

## Notebookサーベイしながら特徴量探し

In [ ]:
# https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm
df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']

#https://www.kaggle.com/dlaststark/gb-vpp-whoppity-dub-dub
df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
# 一番上はU_inout_maxにある
df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
# よくわからん
df['ewm_u_in_mean'] = (df\
                       .groupby('breath_id')['u_in']\
                       .ewm(halflife=9)\
                       .mean()\
                       .reset_index(level=0,drop=True))
df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                          .groupby('breath_id')['u_in']\
                                                          .rolling(window=15,min_periods=1)\
                                                          .agg({"15_in_sum":"sum",
                                                                "15_in_min":"min",
                                                                "15_in_max":"max",
                                                                "15_in_mean":"mean"})\
                                                           .reset_index(level=0,drop=True))